## New Summarizer experiments1

What I need to do

- Retrieve the today news, article from the scrapper database.

- Summarize them with good prompt.

- display them on a UI.

In [ ]:
%load_ext dotenv

In [ ]:
%dotenv ./.env_prod -o

In [ ]:
from src.shared.database import execute_query, generate_database_connection

In [ ]:
yesterday_article_query = "select content, title, posted_at,url from article where posted_at::date = CURRENT_DATE - interval '1 day'"

In [ ]:
from os import getenv

In [ ]:
database_user = getenv('POSTGRES_USER')
database_password = getenv('POSTGRES_PASSWORD')
database_host = getenv('POSTGRES_HOST')
database_port = getenv('POSTGRES_PORT')
database_name = getenv('POSTGRES_DB')

In [ ]:
database_credentials = {
    'user': database_user,
    'password': database_password,
    'host': database_host,
    'port': database_port,
    'database': database_name
}

In [ ]:
connection = generate_database_connection(database_crendentials=database_credentials)

In [ ]:
connection

In [ ]:
results =execute_query(query=yesterday_article_query, database_connection=connection)

In [ ]:
results[0].url

In [ ]:
import pandas as pd

In [ ]:
results_df = pd.DataFrame.from_records(results)

In [ ]:
results_df.columns = ["content", "title", "posted_at", "url"]

In [ ]:
from pathlib import Path

In [ ]:
current_directory = Path.cwd().joinpath("datasets", "today_news")

In [ ]:
current_directory.mkdir(exist_ok=True)

In [ ]:
from datetime import datetime

In [ ]:
today = datetime.now().strftime("%Y-%m-%d")

In [ ]:
results_df.to_csv(current_directory.joinpath(f"{today}-news.csv"))

In [ ]:
results_df.content

In [ ]:
results_df = results_df.drop_duplicates(subset="content")

In [ ]:
results_df.head()

In [ ]:
results_df = results_df.reset_index(drop=True)

In [ ]:
results_df

In [ ]:
from src.rag.components.generator import LLamaCppGeneratorComponent

prompting doesn't seem to wokr, let us use clustering and prompting.

The step will be computing the embedding of the documents, and then cluster the document. Then for each cluster of documents, we will generate a summary.

embedding_model_id = "dunzhang/stella_en_400M_v5"

In [ ]:
embedding_model_id = "dunzhang/stella_en_400M_v5"

In [ ]:
from pathlib import Path

In [ ]:
model_path = Path.cwd().joinpath("models")

In [ ]:
embedding_model_path = model_path.joinpath(embedding_model_id)

In [ ]:

transformer_kwargs = {"model_name_or_path": embedding_model_path.__str__(),
                      "trust_remote_code": True,
                      "device": "cpu",
                      "config_kwargs": {"use_memory_efficient_attention": False,
                                        "unpad_inputs": False},
                      "cache_folder": model_path}

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
sentence_transformer_model = SentenceTransformer(
    **transformer_kwargs)

In [ ]:
today_news_embeddings = sentence_transformer_model.encode(
    results_df.content.tolist())

Not just clustering but let start with paragraph spliting and then embedding.

In [ ]:
today_news_embeddings

In [ ]:
today_news_embeddings.shape

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
axes = plt.figure(figsize=(5, 10))

In [ ]:
similarities = sentence_transformer_model.similarity(today_news_embeddings, today_news_embeddings)

In [ ]:
import seaborn as sb

In [ ]:
#plt.figure(figsize=(50, 50))
#sb.heatmap(similarities, cmap="Blues", annot=True)

In [ ]:
results_df.iloc[25]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import silhouette_score

In [ ]:
def find_best_estimator (X):
    K_mean_estimators = [
        (f"KMeans_{i}", KMeans(n_clusters=i, random_state=42, max_iter=3000)) for i in range(3, X.shape[0])]
    scores = []

    best_estimator = None
    best_metric = float("-inf")
    for estimator_name, estimator in K_mean_estimators :
        estimator.fit(X)
        labels = estimator.labels_
        score = silhouette_score(
            X, labels)
        if score > best_metric :
            best_metric = score
            best_estimator = estimator
        print(estimator_name, score)
        scores.append(score)
    return best_estimator, scores

In [ ]:
best_estimator, scores = find_best_estimator(today_news_embeddings)

In [ ]:
axes = plt.figure(figsize=(5, 10))

In [ ]:
fig, ax = plt.subplots()
ax.plot(range(3, today_news_embeddings.shape[0]), scores)
ax.set_xlabel("number of clusters")
ax.set_ylabel("silhouette score")
ax.set_title("silhouette score vs number of clusters")

In [ ]:
def analyse_embeddings(dataframe, embeddings, index, label_column="labels"):
    """ take an matrix of embeddings, and the labels.
    for each label compute the cosine similarity of document with that label.
    """
    document_in_index = dataframe.query(f"{label_column} == {index}")
    with pd.option_context('display.max_colwidth', None):
        display(document_in_index.title)
    document_index = document_in_index.index
    vectors = embeddings[document_index]
    return sentence_transformer_model.similarity(vectors,  vectors)

In [ ]:
best_estimator.n_clusters

In [ ]:
results_df["labels"] = best_estimator.labels_

In [ ]:
analyse_embeddings(results_df, today_news_embeddings,50)

Actually I just found out that Kmean is good at clustering document, my documents seems to be very noisy, there are a lot of clusters with 1 or 2 documents, and after going through each cluster separately I found that most of the document make sense.

PCA and then clustering

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca_model = PCA(n_components=today_news_embeddings.shape[0])

In [ ]:
reduced_embeddings = pca_model.fit_transform(today_news_embeddings)

In [ ]:
reduced_embeddings.shape

In [ ]:
pca_estimator = find_best_estimator(reduced_embeddings)[0]

In [ ]:
pca_estimator.n_clusters

In [ ]:
pca_estimator.labels_

In [ ]:
best_estimator.labels_

In [ ]:
results_df["labels_pca"] = pca_estimator.labels_

In [ ]:
results_df.loc[results_df.labels_pca != results_df.labels]

In [ ]:
analyse_embeddings(results_df, reduced_embeddings, 3, "labels_pca")

### Applying Hiearchical clustering

We can see that our clusters are good and the first step we were able to separate document, let now try hiearchical clustering to check if there are cluster that can be merged together.

In [ ]:
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import cut_tree

In [ ]:
# Complete Linkage
plt.figure(figsize = (20,10))
mergings = linkage(today_news_embeddings,
                   method='complete', metric='cosine')
dendrogram(mergings)
plt.show()

Also, I found out the dentogram method is working, I will use this post to learn more about it.



https://joernhees.de/blog/2015/08/26/scipy-hierarchical-clustering-and-dendrogram-tutorial/

In [ ]:
# Complete Linkage
plt.figure(figsize=(20, 10))
mergings = linkage(today_news_embeddings,
                   method='complete', metric='cosine')
dendrogram(mergings)
plt.axhline(y=0.45, c='k')
plt.show()

In [ ]:
mergings.shape

In [ ]:
from scipy.cluster.hierarchy import fcluster

label_hierarchical = fcluster(mergings, 0.45, criterion="distance")

In [ ]:
import numpy as np

In [ ]:
results_df.iloc[[20, 78, 7]]

Need to comeback and explore hiearchical clustering

In [ ]:
results_df["labels_hierachical"] = label_hierarchical

In [ ]:
results_df.query("labels_hierachical==22")

Let now find a way to select the best distance cutoff for the hiearacical clustering

The best distance cutoff will be the distance that will give us the maximun shilouette score.

In [ ]:
def select_best(X, merging):
    max_shilouette = float("-inf")
    return_labels = np.zeros(X.shape[0])
    for k in np.arange(0.2, 0.7, 0.01):
        labels = fcluster(merging, k, criterion="distance")
        score = silhouette_score(
            X, labels
        )
        if score > max_shilouette:
            max_shilouette = score
            return_labels = labels
    return max_shilouette, return_labels

In [ ]:
best_distance, label_hierarchical =  select_best(today_news_embeddings, mergings)

In [ ]:
results_df["labels_hierachical"] = label_hierarchical

In [ ]:
analyse_embeddings(results_df, today_news_embeddings, 1, "labels_hierachical")

At this point we have a good clustering but we can go ahead the pic 

Once i have got the best labeling, i can go ahead and select the most important cluster. 

This will be all the cluster with more than 1 document, the rest of the document will be considered as noise. Each cluster with one document will be considered as noise.

In [ ]:
cluster_counts = results_df.labels_hierachical.value_counts()
labels_with_more_than_one = cluster_counts[cluster_counts > 1].index

In [ ]:
important_news_df = results_df.loc[results_df.labels_hierachical.isin(labels_with_more_than_one)]

### Summarization Part.

In [ ]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

In [ ]:
prompt = "You are a french news reporter"

In [ ]:
llama_cpp_generator = LLamaCppGeneratorComponent(
    api_url="http://127.0.0.1:8001",
    model_name=model_name,
    prompt=prompt
)

In [ ]:
llama_cpp_generator._ping_api()

In [ ]:
summarization_prompt = """
Give a summary in french of the following document:
{{content}}
 Describes it in a style of a french new paper reporters.

Don't summarize each document separately, the content in all the documents should be summarized.

The summary should be in french not in English
"""

In [ ]:
first_cluster.content.str.cat(sep="\n")

In [ ]:
llama_cpp_generator.generate_chat_input(
    template_values={"content": first_cluster.content.str.cat(sep="\n")}, prompt_template=summarization_prompt)

In [ ]:
summaries = []
for id, news_group in results_df.groupby("labels"):
    news = news_group.content.str.cat(sep="\n")
    summary = llama_cpp_generator.run(template_values={"content": news}, prompt_template=summarization_prompt)
    summaries.append(summary)
    print(summary)
    print("---" * 10)

I will have to stop here, and then start writing about the learning. 

- Another apporach will be collect the document from radiookapi and fine tune the model.
-   Deploy the solution with streamlit

### Trying Topic Modeling and Then Summarizing

In  this section we will use topic modelling and then summarize the documents in each topic.

Kmean is not giving good results because some cluster are not correct., Let try with the title embeddings.